In [1]:
from skopt import Optimizer
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import RBF, ConstantKernel, Product
from tqdm import tqdm_notebook as tqdm
from skopt import gp_minimize
from time import sleep
import docker

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Глобальные Параметры

In [2]:
# первые n_initial_points модель не обучается
n_initial_points = 5

# число итераций цикла
n_calls = 20

# оптимизация на кубе [low_constraint, high_constraint]^dim
low_constraint, high_constraint = 10., 420.
dim = 2

# столько контейнеров вызываются для параллельной работы
batch_size = 2

# директория на сервере, хранит директории, которые будут монтироваться в контейнеры
folder_local = '/home/matyushinleonid/LHCb ECal/feb_meeting/folder_local'

# директория для файлов input и output внутри контейнера
folder_container = '/home/logs'

# python-клиент докера
client = docker.from_env()

# имя образа
container = "qwaiqir/calosim_build:0.2_python36"

# имена директорий, каждая соответствует своей копии образа
worker_names = ['first_worker', 'second_worker']

### Модель

In [3]:
kernel = Product(ConstantKernel(1), RBF(1)) + ConstantKernel(1)

model = GaussianProcessRegressor(alpha=0, 
                                 normalize_y=True, 
                                 noise='gaussian', 
                                 n_restarts_optimizer=10, 
                                 kernel=kernel)

optimizer = Optimizer([[low_constraint, high_constraint]]*dim,
                      model,
                      n_initial_points=n_initial_points,
                      acq_func='EI',
                      acq_optimizer='lbfgs',
                      random_state=None)

### Оптимизация (+ работа с контейнерами)

In [4]:
def write_input_file(worker_name, input_data):
    file_to_write = '{}/{}/input'.format(folder_local, worker_name)
    string_to_write = ' '.join(map(str, input_data))
    with open(file_to_write, "w") as file:
        print(string_to_write,
              file=file)

def create_job(worker_name):
    folder_to_mount = '{}/{}'.format(folder_local, worker_name)
    client.containers.run(container,
                          remove=True,
                          detach=True,
                          volumes={folder_to_mount: {'bind': folder_container,
                                                     'mode': 'rw'}})

def read_output_file(worker_name):
    file_to_read = '{}/{}/output'.format(folder_local, worker_name)
    with open(file_to_read, 'r') as myfile:
        data = myfile.read()
    return float(data)

In [5]:
for i in tqdm(range(n_calls)):
    X = optimizer.ask(n_points=batch_size)
    for i, worker_name in enumerate(worker_names):
        x = X[i]
        write_input_file(worker_name, x)
        
        #собственно, тут должен быть запуск контейнера
        #create_job(worker_name)
    
    sleep(5)
    
    Y = []
    for i, worker_name in enumerate(worker_names):
        y = read_output_file(worker_name)
        Y.append(y)
        
    optimizer.tell(X, Y)

Widget Javascript not detected.  It may not be installed or enabled properly.
